# Get Pretrained Models
Okay

In [15]:
!mkdir -p checkpoints/

!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt -P checkpoints

--2025-04-30 18:31:27--  https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt
Resolving docs-assets.developer.apple.com (docs-assets.developer.apple.com)... 17.253.73.202, 17.253.73.201
Connecting to docs-assets.developer.apple.com (docs-assets.developer.apple.com)|17.253.73.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215934653 (206M) [application/octet-stream]
Saving to: ‘checkpoints/mobileclip_s0.pt’

mobileclip_s0.pt    100%[===================>] 205,93M  5,49MB/s    in 39s     

2025-04-30 18:32:07 (5,34 MB/s) - ‘checkpoints/mobileclip_s0.pt’ saved [215934653/215934653]



# Create the models

In [ ]:
!pip install torch
!pip install torchvision
!pip install timm
!pip install open-clip-torch
!pip install datasets
!pip install clip-benchmark

# Model Testing

In [1]:
import torch
from PIL import Image
import mobileclip

model, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/checkpoints/mobileclip_s0.pt')
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

image = preprocess(Image.open("pngwing.com(1).png").convert('RGB')).unsqueeze(0)
text = tokenizer(["a purple dog", "a white dog", "a black dog"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Set the print options for PyTorch to avoid scientific notation and limit decimal places
torch.set_printoptions(sci_mode=False, precision=4)

print("Label probs:", text_probs)

/opt/anaconda3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/pngwing.com(1).png'

# Changing the Positional Embeddings

In [2]:
print(model.get_positional_embedding() )

def get_positional_embedding(self, lambda2: int = 4):
    """
    Get modified positional embedding for text encoder based on the given formula.
    """
    pos_embed = self.text_encoder.get_positional_embedding().pos_embed.pos_embed
    if pos_embed is None:
        raise ValueError("Positional embedding not found in text encoder.")

    max_pos, embed_dim = pos_embed.shape[2], pos_embed.shape[3]
    modified_pos_embed = torch.zeros((1, 1, max_pos, embed_dim), device=pos_embed.device)

    for pos in range(max_pos):
        if pos <= 20:
            modified_pos_embed[:, :, pos, :] = pos_embed[:, :, pos, :]
        else:
            lower_idx = pos // lambda2
            upper_idx = min(lower_idx + 1, max_pos - 1)  # Ensure upper_idx is within bounds
            alpha = (pos % lambda2) / lambda2
            modified_pos_embed[:, :, pos, :] = (1 - alpha) * pos_embed[:, :, lower_idx, :] + alpha * pos_embed[:, :, upper_idx, :]
    # turn the torch tensor into nn parameter
    modified_pos_embed = torch.nn.Parameter(modified_pos_embed, requires_grad=False)
    return modified_pos_embed

# Example usage
lambda2 = 4
new_pos_embed = get_positional_embedding(model, lambda2)
print("Modified Positional Embedding:", new_pos_embed)

# set the models pos embedding to the new one
model.text_encoder.get_positional_embedding().pos_embed.pos_embed = new_pos_embed


LearnablePositionalEmbedding(num_embeddings=77, embedding_dim=512, padding_idx=None)
Modified Positional Embedding: Parameter containing:
tensor([[[[     0.0000,      0.0000,      0.0000,  ...,      0.0000,
                0.0000,      0.0000],
          [     0.0041,      0.0016,     -0.0007,  ...,      0.0007,
                0.0036,     -0.0076],
          [     0.0077,      0.0026,      0.0012,  ...,      0.0001,
                0.0013,     -0.0043],
          ...,
          [     0.0075,      0.0051,      0.0000,  ...,      0.0011,
                0.0001,     -0.0003],
          [     0.0031,      0.0068,     -0.0008,  ...,      0.0024,
               -0.0002,     -0.0025],
          [    -0.0013,      0.0086,     -0.0016,  ...,      0.0036,
               -0.0004,     -0.0046]]]])


# Testing the model after changing the positional embeddings

In [10]:
image = preprocess(Image.open("pngwing.com(1).png").convert('RGB')).unsqueeze(0)
text = tokenizer(["a purple dog", "a white dog", "a black dog"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Set the print options for PyTorch to avoid scientific notation and limit decimal places
torch.set_printoptions(sci_mode=False, precision=4)

print("Label probs:", text_probs)

Label probs: tensor([[0.7638, 0.0161, 0.2201]])


# Downloading the captioned images


In [3]:
# --- Configuration ---
import os

# Using relative paths for better portability
# This creates a 'data' directory in the project folder
BASE_DATA_DESTINATION = os.path.join(os.getcwd(), "data")
KAGGLE_FLICKR8K_URL = "https://www.kaggle.com/api/v1/datasets/download/adityajn105/flickr8k"
FLICKR8K_ZIP_FILENAME = "flickr8k.zip"
FLICKR8K_IMAGES_FOLDER_NAME = "Images"
CAPTIONS_CSV_FILENAME = "captions.csv"
OUTPUT_FOLDER_NAME = "output"

import zipfile
import requests
import io

def download_file(url: str, destination_path: str):
    print(f"Downloading from {url} to {destination_path}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(destination_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print("Download complete.")
    except Exception as e:
        print(f"Error: {e}")
        raise

def extract_zip_file(zip_path: str, destination_folder: str):
    print(f"Extracting {zip_path} to {destination_folder}...")
    try:
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(destination_folder)
        print("Extraction complete.")
    except Exception as e:
        print(f"Extraction error: {e}")
        raise

def setup_data_directory(base_data_path: str):
    images_path = os.path.join(base_data_path, FLICKR8K_IMAGES_FOLDER_NAME)
    output_path = os.path.join(base_data_path, OUTPUT_FOLDER_NAME)
    os.makedirs(base_data_path, exist_ok=True)
    os.makedirs(output_path, exist_ok=True)
    return base_data_path, images_path, output_path


print(f"Setting up data directories in: {BASE_DATA_DESTINATION}")
base_dir, images_dir, output_dir = setup_data_directory(BASE_DATA_DESTINATION)

zip_file_path = os.path.join(base_dir, FLICKR8K_ZIP_FILENAME)

if not os.path.exists(images_dir):
    print("Images not found. Attempting download...")
    try:
        download_file(KAGGLE_FLICKR8K_URL, zip_file_path)
        extract_zip_file(zip_file_path, base_dir)
        os.remove(zip_file_path)
    except Exception as e:
        print(f"Failed to set up dataset: {e}")
        raise FileNotFoundError(f"Please manually download and extract to: {base_dir}")
else:
    print(f"Images already exist at {images_dir}.")

Setting up data directories in: /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data
Images not found. Attempting download...
Download complete.
Extracting /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/flickr8k.zip to /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data...
Extraction complete.


# Train the model using the downloaded images and custom captions

Importing the required libraries
setting parameters and lookups


In [6]:
import torch
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
import os
import json
from PIL import Image
from tqdm import tqdm
import mobileclip
import random

# --- Configuration ---
# Using relative paths for better portability
BASE_DATA_DESTINATION = os.path.join(os.getcwd(), "data")
FLICKR8K_IMAGES_FOLDER_NAME = "Images"
CAPTIONS_JSON_FILENAME = "captions.json"
CHECKPOINT_DIR = os.path.join(os.getcwd(), "checkpoints")
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MobileCLIP model
model_path = os.path.join(CHECKPOINT_DIR, 'mobileclip_s0.pt')
model, _, preprocess = mobileclip.create_model_and_transforms(
    'mobileclip_s0', 
    pretrained=model_path
)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')
model.to(DEVICE)

CLIP(
  (image_encoder): MCi(
    (model): FastViT(
      (patch_embed): Sequential(
        (0): MobileOneBlock(
          (se): Identity()
          (activation): GELU(approximate='none')
          (reparam_conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        )
        (1): MobileOneBlock(
          (se): Identity()
          (activation): GELU(approximate='none')
          (reparam_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64)
        )
        (2): MobileOneBlock(
          (se): Identity()
          (activation): GELU(approximate='none')
          (reparam_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (network): ModuleList(
        (0): Sequential(
          (0): RepMixerBlock(
            (token_mixer): RepMixer(
              (reparam_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
            )
            (convffn): ConvFFN(
              (con

## Load the dataset

In [16]:
# Custom Dataset for Flickr8k with the specific JSON caption format
class Flickr8kCaptionedDataset(Dataset):
    def __init__(self, image_dir, captions_file, preprocess_fn, pull_from_json=True):
        self.image_dir = image_dir
        self.preprocess_fn = preprocess_fn
        
    
        # Create list of samples
        self.samples = []
        
        if pull_from_json:
            # Load captions from JSON file
            with open(captions_file, 'r') as f:
                self.captions_data = json.load(f)
            
            # Process JSON with format {"image.jpg": {"long_caption": "...", "short_caption": "..."}, ...}
            for image_name, captions in self.captions_data.items():
                if "long_caption" in captions and "short_caption" in captions:
                    # Add both caption types for each image
                    self.samples.append((image_name, captions["long_caption"], "long"))
                    self.samples.append((image_name, captions["short_caption"], "short"))
        else:
            # Use the default Flickr8k captions file
            captions_file = "data/captions.txt"
            with open(captions_file, 'r') as f:
                lines = f.readlines()
                
            # Process the standard Flickr8k format 
            # Typically each line has format: "image_name#caption" or "image_name,caption"
            for line in lines:
                line = line.strip()
                if line:
                    # Try to split by common delimiters
                    if '#' in line:
                        parts = line.split('#', 1)
                    else:
                        parts = line.split(',', 1)
                        
                    if len(parts) == 2:
                        image_name, caption = parts
                        # Since we don't have long/short distinction, mark as "standard"
                        self.samples.append((image_name.strip(), caption.strip(), "standard"))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        image_name, caption, caption_type = self.samples[idx]
        image_path = os.path.join(self.image_dir, image_name)
        
        # Load and preprocess the image
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.preprocess_fn(image)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Return a random valid sample instead
            return self.__getitem__(random.randint(0, len(self) - 1))
        
        return image, caption, caption_type


## Loss function

In [8]:
# Contrastive Loss Function
def contrastive_loss(image_embeds, text_embeds, temperature=0.07):
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=1)
    text_embeds = F.normalize(text_embeds, dim=1)
    
    # Compute similarity matrix
    logits = torch.matmul(image_embeds, text_embeds.T) / temperature
    
    # Labels are the positions of the positive pairs
    labels = torch.arange(logits.size(0), device=logits.device)
    
    # Compute loss in both directions (image->text and text->image)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.T, labels)
    
    return (loss_i2t + loss_t2i) / 2

## Training the model

In [14]:
# Set up dataset and dataloader
images_dir = os.path.join(BASE_DATA_DESTINATION, FLICKR8K_IMAGES_FOLDER_NAME)
captions_file = os.path.join(BASE_DATA_DESTINATION, CAPTIONS_JSON_FILENAME)

pull_from_json = True

# Check if files exist
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Images directory not found: {images_dir}")
if not os.path.exists(captions_file):
    print(f"Captions file not found: {captions_file}")
    pull_from_json = False

# Create dataset and dataloader
dataset = Flickr8kCaptionedDataset(images_dir, captions_file, preprocess, pull_from_json=pull_from_json)
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=0,
    drop_last=True
)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
def train():
    model.train()
    
    for epoch in range(NUM_EPOCHS):
        total_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        
        for batch_idx, (images, captions, caption_types) in enumerate(progress_bar):
            images = images.to(DEVICE)
            
            # Tokenize the captions
            tokenized_captions = tokenizer(captions).to(DEVICE)
            
            # Forward pass with mixed precision
            with torch.cuda.amp.autocast():
                image_features = model.encode_image(images)
                text_features = model.encode_text(tokenized_captions)
                
                # Compute contrastive loss
                loss = contrastive_loss(image_features, text_features)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Update progress bar
            total_loss += loss.item()
            avg_loss = total_loss / (batch_idx + 1)
            progress_bar.set_postfix(loss=f"{avg_loss:.4f}")
            
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.4f}")
        
        # Save checkpoint
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"mobileclip_finetuned_epoch{epoch+1}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved: {checkpoint_path}")



Captions file not found: /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/captions.json


## Running the training loop

In [ ]:
# Run training
print(f"Training on device: {DEVICE}")
print(f"Dataset size: {len(dataset)} samples")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Number of epochs: {NUM_EPOCHS}")

train()

# Evaluate model after training
model.eval()

# Example evaluation
test_image_path = "pngwing.com(1).png"
test_texts = ["a purple dog", "a white dog", "a black dog"]

test_image = preprocess(Image.open(test_image_path).convert('RGB')).unsqueeze(0).to(DEVICE)
test_text = tokenizer(test_texts).to(DEVICE)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(test_image)
    text_features = model.encode_text(test_text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

torch.set_printoptions(sci_mode=False, precision=4)
print("Label probabilities after training:", text_probs)

Training on device: cpu
Dataset size: 40456 samples
Batch size: 32
Learning rate: 0.0001
Number of epochs: 10


Epoch 1/10:   1%|          | 14/1264 [02:28<3:28:56, 10.03s/it, loss=0.7555]

Error loading image /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/Images/2924483864_cfdb900a13.jpg,White dog with yellow and black: [Errno 2] No such file or directory: '/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/Images/2924483864_cfdb900a13.jpg,White dog with yellow and black'


Epoch 1/10:   2%|▏         | 31/1264 [05:30<3:52:37, 11.32s/it, loss=0.6701]

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Create the dataset and dataloader
train_dataset = CustomDataset(cifar10_dataset, transform=preprocess)
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Replace `MobileCLIPModel()` with your Mobile CLIP implementation.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 3. Contrastive Loss (InfoNCE)
def contrastive_loss(image_embeds: torch.Tensor, text_embeds: torch.Tensor, temperature: float = 0.07) -> torch.Tensor:
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=1)
    text_embeds = F.normalize(text_embeds, dim=1)
    # Compute logits
    logits = image_embeds @ text_embeds.t() / temperature
    labels = torch.arange(logits.size(0), device=logits.device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.t(), labels)
    return (loss_i2t + loss_t2i) / 2

# 4. Optimizer and training setup
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 10

# 5. Training loop
def train():
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, captions in tqdm(dataloader):
            images = images.to(device)
            captions = tokenizer(captions).to(device)
            # Forward pass: user-provided model should return (image_embeds, text_embeds)
            image_embeds = model.encode_image(images)
            text_embeds = model.encode_text(captions)

            # Compute contrastive loss
            loss = contrastive_loss(image_embeds, text_embeds)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * images.size(0)

        avg_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

if __name__ == '__main__':
    train()


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# Define the transformation pipeline
# Remove transforms.ToPILImage() since CIFAR-10 images are already PIL images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Load the CIFAR-10 dataset
cifar10_dataset = datasets.CIFAR10(root='cifar10', train=True, download=True)

# Apply the transformation pipeline to the images
images = [transform(img) for img, _ in cifar10_dataset]

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
model.eval()

# Generate captions for a subset (e.g., first 100 images for demo)
captions = []
for image in images[:100]:  # Increase this number as needed
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    captions.append(caption)

# Custom dataset combining image and generated caption
class CIFAR10WithCaptions(Dataset):
    def __init__(self, images, captions, transform=None):
        self.images = images
        self.captions = captions
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        caption = self.captions[idx]
        if self.transform:
            img = self.transform(img)
        return img, caption

# Example usage
transform_tensor = transforms.ToTensor()
custom_dataset = CIFAR10WithCaptions(images[:100], captions, transform=transform_tensor)


Files already downloaded and verified


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-2.1179039478302, 2.393557548522949] which cannot be converted to uint8.

In [ ]:

# Example usage
transform_tensor = transforms.ToTensor()
custom_dataset = CIFAR10WithCaptions(images[:100], captions, transform=transform_tensor)

NameError: name 'CIFAR10WithCaptions' is not defined

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

# Get the first image and caption
img, caption = custom_dataset[0]

# Convert the tensor image to a PIL image for plotting
img = F.to_pil_image(img)

# Plot the image
plt.imshow(img)
plt.title(f"Caption: {caption}")
plt.axis('off')
plt.show()

NameError: name 'custom_dataset' is not defined